In [1]:
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

df_krx = fdr.StockListing('KRX')
print(len(df_krx))
df_krx.head()

2873


,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,56900,2,-400,-0.70,56600,57000,56500,3142778,178227566700,339680627095000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,199900,2,-5100,-2.49,199800,201000,199200,722355,144309615400,145527672763500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,376500,2,-9000,-2.33,386000,386000,376000,38691,14697257000,88101000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,1180000,1,29000,2.52,1142000,1187000,1140000,29801,34991961000,83985320000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,204500,2,-2500,-1.21,205000,205000,203500,91071,18621845000,42825611059500,209416191,STK


In [2]:
df_KOSPI = fdr.DataReader('KS11', '2024')   # KOSPI
df_Target = fdr.DataReader('005930', '2024')  # Target 종목 : 삼성전자

df_KOSPI.head(10)

1 001 2024-01-01 00:00:00 2025-12-31 00:00:00


,Open,High,Low,Close,Volume,Change,UpDown,Comp,Amount,MarCap
Date,,,,,,,,,,
2024-01-02,2645.47,2675.80,2641.88,2669.81,409871688,0.0055,1,14.53,9628190129018,2147223894556881
2024-01-03,2643.54,2643.72,2607.31,2607.31,463131608,-0.0234,2,-62.50,10121578328315,2096573473823721
2024-01-04,2592.44,2602.64,2580.09,2587.02,770176172,-0.0078,2,-20.29,8992273956601,2081862659594752
2024-01-05,2586.89,2592.29,2572.60,2578.08,522289892,-0.0035,2,-8.94,8384472928787,2075148802709824
2024-01-08,2584.23,2591.68,2566.34,2567.82,320143990,-0.0040,2,-10.26,6763632029996,2067290656237408
2024-01-09,2598.31,2599.37,2556.00,2561.24,791214273,-0.0026,2,-6.58,8896714487368,2061575745648289
2024-01-10,2563.97,2568.19,2539.82,2541.98,570021066,-0.0075,2,-19.26,8795834820834,2045533566125824
2024-01-11,2543.03,2557.30,2540.27,2540.27,788422596,-0.0007,2,-1.71,13669889824902,2043896294511727
2024-01-12,2536.55,2543.83,2517.76,2525.05,790177187,-0.0060,2,-15.22,8368765827550,2047248114238733


In [4]:
# 동일 카테고리에서 비교군의 설정
object = ['KS11', '105560', '316140', '086790', '024110']    # 금융지주사 비교

for i in range(len(object)):
    print(str(df_krx[df_krx['Code'] == object[i]]['Name'].values).replace('\'', ''))

# plot legend에 한글 문자열 전달 시 발생하는 기호 제거 방법 (250221) 
result = []

for i in range(len(object)):
    result.append(str(df_krx[df_krx['Code'] == object[i]]['Name'].values).replace('\'', '').replace('[', '').replace(']', ''))

result


[]
[KB금융]
[우리금융지주]
[하나금융지주]
[기업은행]


['', 'KB금융', '우리금융지주', '하나금융지주', '기업은행']

In [5]:
# plot에서 한글 폰트 깨짐 문제

import platform

os = platform.system()

# Win
if os == 'Windows':
    plt.rc('font', family = 'Malgun Gothic')

# Mac
elif os == 'Darwin':
    plt.rc('font', family = 'AppleGothic')